In [1]:
import os
import numpy as np
import pickle

# ===========================
# 基本配置
# ===========================
data_dir = "data_processed"        # EEG 数据所在目录
save_dir = "data_sequences"        # 输出目录
os.makedirs(save_dir, exist_ok=True)

segment_len = 6     # 每个动作段包含 6 个 trial
stride = 6          # 不重叠
datasets = [
    "val_ood_2subj.npz",
    "test_id_5subj.npz",
    "test_ood_2subj.npz"
]

# ===========================
# 函数：生成动作段
# ===========================
def make_segments(X, y, L=6, stride=6):
    """
    按照动作长度 L 生成 trial 段
    X: (N, 22, 1000)
    y: (N,)
    返回：
      segments: (num_segments, L, 22, 1000)
      labels:   (num_segments, L)
    """
    num_segments = (len(X) - L) // stride + 1
    segments, labels = [], []

    for i in range(0, num_segments * stride, stride):
        seg = X[i:i + L]
        lab = y[i:i + L]
        if len(seg) == L:
            segments.append(seg)
            labels.append(lab)

    segments = np.array(segments)
    labels = np.array(labels)
    return segments, labels


# ===========================
# 主流程
# ===========================
summary = {}

for fname in datasets:
    path = os.path.join(data_dir, fname)
    data = np.load(path)
    X, y = data["X"], data["y"]

    segs, labs = make_segments(X, y, L=segment_len, stride=stride)

    base = os.path.splitext(fname)[0]
    save_path = os.path.join(save_dir, f"{base}_segments.pkl")
    with open(save_path, "wb") as f:
        pickle.dump({"segments": segs, "labels": labs}, f)

    summary[base] = {
        "input_trials": len(X),
        "segments": len(segs),
        "segment_shape": list(segs.shape)
    }

    print(f" Saved {base}_segments.pkl: {segs.shape} ({len(segs)} segments)")


print("\n=== Summary ===")
for k, v in summary.items():
    print(f"{k}: {v['input_trials']} trials → {v['segments']} segments, shape={v['segment_shape']}")


 Saved val_ood_2subj_segments.pkl: (96, 6, 22, 1000) (96 segments)
 Saved test_id_5subj_segments.pkl: (36, 6, 22, 1000) (36 segments)
 Saved test_ood_2subj_segments.pkl: (96, 6, 22, 1000) (96 segments)

=== Summary ===
val_ood_2subj: 576 trials → 96 segments, shape=[96, 6, 22, 1000]
test_id_5subj: 220 trials → 36 segments, shape=[36, 6, 22, 1000]
test_ood_2subj: 576 trials → 96 segments, shape=[96, 6, 22, 1000]
